In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd

In [2]:
dataTrain=pd.read_csv("../data/newtrain.csv")
dataTest=pd.read_csv("../data/newtest.csv")
print(dataTrain.head())
print(dataTrain.shape)

   Unnamed: 0  round  venue   gf   ga   xg  xga  poss  formation    sh  ...  \
0           0     33      0  0.0  0.0  1.4  0.8  49.0          4  11.0  ...   
1           1     35      1  2.0  0.0  2.4  0.4  64.0          4  25.0  ...   
2           2     37      0  5.0  2.0  3.5  1.0  67.0          4  30.0  ...   
3           3      2      0  1.0  3.0  0.3  1.8  43.0          4   6.0  ...   
4           4      3      1  2.0  0.0  2.1  0.7  43.0          4  14.0  ...   

   ga_rolling  sh_rolling  sot_rolling  dist_rolling  fk_rolling  pk_rolling  \
0    2.333333   17.333333     6.333333     17.300000    0.333333         0.0   
1    1.333333   12.000000     3.666667     16.566667    0.333333         0.0   
2    0.000000   17.666667     6.333333     17.166667    1.000000         0.0   
3    0.666667   22.000000     8.000000     17.500000    1.333333         0.0   
4    1.666667   20.333333     8.333333     18.500000    1.666667         0.0   

   pkatt_rolling  home_code  target_away  ta

In [3]:
del dataTrain["Unnamed: 0"]
del dataTest["Unnamed: 0"]
del dataTrain["target_away"]
del dataTest["target_away"]

In [4]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   round          1806 non-null   int64  
 1   venue          1806 non-null   int64  
 2   gf             1806 non-null   float64
 3   ga             1806 non-null   float64
 4   xg             1806 non-null   float64
 5   xga            1806 non-null   float64
 6   poss           1806 non-null   float64
 7   formation      1806 non-null   int64  
 8   sh             1806 non-null   float64
 9   sot            1806 non-null   float64
 10  dist           1806 non-null   float64
 11  fk             1806 non-null   float64
 12  pk             1806 non-null   float64
 13  pkatt          1806 non-null   float64
 14  venue_code     1806 non-null   int64  
 15  opp_code       1806 non-null   int64  
 16  hour           1806 non-null   int64  
 17  day_code       1806 non-null   int64  
 18  gf_rolli

In [5]:
# Data Training
train_data = dataTrain.iloc[:, :-1].values
train_labels = dataTrain.iloc[:, -1:].values

# Data Testing
test_data = dataTest.iloc[:, :-1].values
test_labels = dataTest.iloc[:, -1:].values

In [6]:
# Normalisasi Data
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Pembentukan Data Sequence
sequence_length = 27  # Panjang sequence yang akan digunakan
num_features = train_data.shape[1]

In [9]:
def create_sequences(data, labels, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length + 1):
        X.append(data[i:i+sequence_length])
        y.append(labels[i+sequence_length-1])
    return np.array(X), np.array(y)

In [18]:
X_train, y_train = create_sequences(train_data, train_labels, sequence_length)
X_test, y_test = create_sequences(test_data, test_labels, sequence_length)

In [21]:
# Pembangunan Model LSTM
model = Sequential()
model.add(LSTM(units=64, input_shape=(sequence_length, num_features)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [22]:
# Pelatihan Model
model.fit(X_train, y_train, epochs=10, batch_size=32)  # Sesuaikan dengan jumlah epoch dan ukuran batch yang optimal

Epoch 1/10


56/56 [==============================] - 7s 124ms/step - loss: 0.6367 - accuracy: 0.6511
Epoch 2/10
56/56 [==============================] - 7s 121ms/step - loss: 0.5915 - accuracy: 0.6972
Epoch 3/10
56/56 [==============================] - 7s 127ms/step - loss: 0.5422 - accuracy: 0.7337
Epoch 4/10
56/56 [==============================] - 8s 138ms/step - loss: 0.4916 - accuracy: 0.7764
Epoch 5/10
56/56 [==============================] - 7s 127ms/step - loss: 0.4470 - accuracy: 0.8146
Epoch 6/10
56/56 [==============================] - 7s 127ms/step - loss: 0.4080 - accuracy: 0.8354
Epoch 7/10
56/56 [==============================] - 7s 127ms/step - loss: 0.3760 - accuracy: 0.8461
Epoch 8/10
56/56 [==============================] - 9s 165ms/step - loss: 0.3413 - accuracy: 0.8702
Epoch 9/10
56/56 [==============================] - 9s 156ms/step - loss: 0.3164 - accuracy: 0.8798
Epoch 10/10
55/56 [============================>.] - ETA: 0s - loss: 0.2953 - accuracy: 0.8943ERROR:tensorflow:

In [23]:
# Testing Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")

19/19 [==============================] - 1s 70ms/step - loss: 0.3032 - accuracy: 0.8748
Loss: 0.3032
Accuracy: 0.8748


In [24]:
from sklearn.model_selection import KFold

# Tentukan jumlah lipatan (folds) untuk cross validation
num_folds = 5

# Buat objek KFold untuk melakukan cross validation
kf = KFold(n_splits=num_folds, shuffle=True)

# List untuk menyimpan skor akurasi pada setiap lipatan
scores = []

# Melakukan cross validation
for train_index, val_index in kf.split(X_train):
    # Bagi data menjadi data pelatihan dan data validasi pada lipatan saat ini
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    # Reset ulang model sebelum setiap pelatihan
    model.reset_states()
    
    # Pelatihan Model pada lipatan saat ini
    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32)
    
    # Evaluasi model pada data validasi lipatan saat ini
    _, accuracy = model.evaluate(X_val_fold, y_val_fold)
    
    # Menyimpan skor akurasi pada lipatan saat ini
    scores.append(accuracy)

# Tampilkan skor akurasi pada setiap lipatan
for fold, score in enumerate(scores):
    print(f"Fold {fold+1}: {score:.4f}")

# Tampilkan rata-rata dan standar deviasi dari skor akurasi
print(f"Mean Accuracy: {np.mean(scores):.4f}")
print(f"Standard Deviation: {np.std(scores):.4f}")


Epoch 1/10
45/45 [==============================] - 5s 114ms/step - loss: 0.2720 - accuracy: 0.9066
Epoch 2/10
45/45 [==============================] - 5s 117ms/step - loss: 0.2540 - accuracy: 0.9129
Epoch 3/10
45/45 [==============================] - 5s 113ms/step - loss: 0.2399 - accuracy: 0.9199
Epoch 4/10
45/45 [==============================] - 6s 132ms/step - loss: 0.2303 - accuracy: 0.9192
Epoch 5/10
45/45 [==============================] - 5s 114ms/step - loss: 0.2111 - accuracy: 0.9438
Epoch 6/10
45/45 [==============================] - 6s 130ms/step - loss: 0.2004 - accuracy: 0.9389
Epoch 7/10
45/45 [==============================] - 6s 144ms/step - loss: 0.1871 - accuracy: 0.9473
Epoch 8/10
45/45 [==============================] - 6s 139ms/step - loss: 0.1796 - accuracy: 0.9579
Epoch 9/10
45/45 [==============================] - 6s 137ms/step - loss: 0.1716 - accuracy: 0.9522
Epoch 10/10
12/12 [==============================] - 1s 72ms/step - loss: 0.1702 - accuracy: 0.9494


In [25]:
X

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1806 entries, 0 to 1805
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   round          1806 non-null   int64  
 1   venue          1806 non-null   int64  
 2   gf             1806 non-null   float64
 3   ga             1806 non-null   float64
 4   xg             1806 non-null   float64
 5   xga            1806 non-null   float64
 6   poss           1806 non-null   float64
 7   formation      1806 non-null   int64  
 8   sh             1806 non-null   float64
 9   sot            1806 non-null   float64
 10  dist           1806 non-null   float64
 11  fk             1806 non-null   float64
 12  pk             1806 non-null   float64
 13  pkatt          1806 non-null   float64
 14  venue_code     1806 non-null   int64  
 15  opp_code       1806 non-null   int64  
 16  hour           1806 non-null   int64  
 17  day_code       1806 non-null   int64  
 18  gf_rolli

In [27]:
data = [0, 0, 2, 0, 1, 1.2, 44, 15, 10, 2, 14.6, 1, 0, 0, 0, 6, 20, 4, 1.666666667, 2, 15, 5, 20.7, 0, 0.3333333333, 0.3333333333, 0]
data_array = np.array(data)


In [29]:
normalized_data = scaler.transform(data_array.reshape(1, -1))


In [30]:
X_test, y_test = create_sequences(normalized_data, [], sequence_length)

In [31]:
# # Melakukan prediksi pada data uji
# y_pred = model.predict(X_test)

# # Melakukan invers normalisasi pada hasil prediksi
# y_pred_inverse = scaler.inverse_transform(y_pred)

# # Menampilkan hasil prediksi
# print("Hasil Prediksi:")
# for i in range(len(X_test)):
#     print(f"Data: {X_test[i]}, Prediksi: {y_pred_inverse[i]}")


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.